In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"], axis=1)
  y = data[["suggestion"]]
  y = pd.get_dummies(y)
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2,random_state=99)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        #print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 10 == 0:
           print(f"{files.index(file)+1+index}",end=" ")
           if (files.index(file)+index+1) % 400 == 0:
             print(" ")

        address = f"/content/data/{file}"
        try:
           unattached_dfs.append(process(pd.read_csv(address),min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that))
        except:
          pass
     nud = []
     for i in unattached_dfs:
       if i.size != 0:
         nud.append(i)
     data = np.concatenate(nud)
     data = pd.DataFrame(data, columns=clmns)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 0 
  for files in new_files:
     each_file_proc(files,now,index,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that)
     index = index + len(files)
  print(" ")
  return now


 
def scaler(row,min_range,max_range):
    scaler = MinMaxScaler(feature_range=(min_range, max_range))
    row = scaler.fit_transform(row)
    return row
def make_y(data, i,how_many_future_candles,how_many_past_candles):
  mean_of_next = []
  for ix in range(i,i+how_many_future_candles):
    mean_of_next.append((data[ix][3]-data[ix][0]))
  mean_of_next = sum(mean_of_next)
  mean_of_previous = []
  for ix in range(i-how_many_past_candles,i):
    mean_of_previous.append((data[ix][3]-data[ix][0]))
  mean_of_previous = sum(mean_of_previous)

  if data[i-1][3] < data[i+4][0]:
    return "buy"
  else:
    return "sell"
def process(data,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(each_row_past, data.shape[0]-10):
        opens = []
        highs = []
        lows = []
        closes = []           
        for ix in range(each_row_past, 1,-1):
            opens.append([data[i-ix][0] - data[i-ix+1][0] ])
            closes.append([data[i-ix][3] - data[i-ix+1][3] ])
        
        opens = scaler(np.array(opens).reshape(-1, 1),min_range,max_range)
        closes = scaler(np.array(closes).reshape(-1, 1),min_range,max_range)
        
        sugg = make_y(data, i,how_many_future_candles,how_many_past_candles)
        arr = np.concatenate((np.array(opens).flatten(), np.array(highs).flatten()
        , np.array(lows).flatten(), np.array(closes).flatten()), axis=0).reshape(-1, 1)
        #arr = scaler(arr.reshape(-1, 1),min_range,max_range)
        arr = np.append(arr, sugg)
        row.append(arr)
    return np.array(row)

In [15]:
clmns = list(range(1, 99))
clmns.append("suggestion")

min_range,max_range = -1,1
how_many_future_candles = 5
how_many_past_candles = 5
each_row_past,interval_for_that = 50,-1

into_how_many_csv = 100
csv_to_parquet = 10

#xTrain,xTest,yTrain,yTest = start(min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that,into_how_many_csv,csv_to_parquet)
#print(yTrain.value_counts())
#print(yTest.value_counts())

In [31]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"max","1mo")

Symbols : 1500
Data Folder Removed
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [17]:
data = yf.download("aapl",period="max",interval="1d")

[*********************100%***********************]  1 of 1 completed


In [28]:
lol = []
data = np.array(data)
for i in range(each_row_past, data.shape[0]-10):
   er = []
   for ix in range(each_row_past, 0,-1):
       er.append(data[i-ix])
   lol.append(er)

In [29]:
np.array(lol).shape

(10417, 50, 6)

In [30]:
pd.DataFrame(lol[0])

,0,1,2,3,4,5
0,0.128348,0.128906,0.128348,0.128348,0.100178,469033600.0
1,0.122210,0.122210,0.121652,0.121652,0.094952,175884800.0
2,0.113281,0.113281,0.112723,0.112723,0.087983,105728000.0
3,0.115513,0.116071,0.115513,0.115513,0.090160,86441600.0
4,0.118862,0.119420,0.118862,0.118862,0.092774,73449600.0
5,0.126116,0.126674,0.126116,0.126116,0.098436,48630400.0
6,0.132254,0.132813,0.132254,0.132254,0.103227,37363200.0
7,0.137835,0.138393,0.137835,0.137835,0.107583,46950400.0
8,0.145089,0.145647,0.145089,0.145089,0.113245,48003200.0
9,0.158482,0.159040,0.158482,0.158482,0.123699,55574400.0


In [36]:
lol = np.array(lol)

In [42]:
lol.shape

(10417, 50, 6)

In [38]:
xTrain = lol[:9000]

In [53]:
xTrain.shape

(9000, 50, 6)

In [40]:
xTest = lol[9000:]

In [41]:
xTest.shape

(1417, 50, 6)

In [93]:
yTrain = []
for i in xTrain:
  if i[49][3] > i[44][3]:
    yTrain.append(1)
  else:
    yTrain.append(0)


In [94]:
yTrain = np.array(yTrain)

In [95]:
yTrain.shape

(9000,)

In [48]:
yTest = []
for i in xTest:
  if i[49][3] > i[44][3]:
    yTest.append(1)
  else:
    yTest.append(0)


In [49]:
yTest = np.array(yTest)
yTest.shape

(1417,)

In [64]:
yTrain.shape

(9000,)

In [65]:
yTrain.reshape(1,-1)

array([[0, 0, 0, ..., 1, 0, 0]])

In [82]:
yTrain = np.expand_dims(yTrain,axis=0)

In [86]:
yTrain.shape

(9000, 1)

In [87]:
yTrain = yTrain.reshape(-1,1,1)

In [88]:
yTrain.shape

(9000, 1, 1)

In [98]:
model = Sequential()

model.add(Dense(750, activation='relu', input_shape=(50,6)))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Flatten())
model.add(Dense(750, activation='relu'))


model.add(Dense(1, activation='sigmoid'))

opt = tf.keras.optimizers.Adamax()

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

NameError: ignored

In [97]:
model.fit(xTrain,yTrain,epochs=15,batch_size=32)
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"AL6W_{acr}.h5")

Epoch 1/15


ValueError: ignored

In [ ]:
def process_for_prediction(data,index,min_range,max_range,each_row_past,interval_for_that):
        i = index
        if "symbol" in data.columns:
              data.drop("symbol",axis=1,inplace=True) 
        if "datetime" in data.columns:
              data.drop("datetime",axis=1,inplace=True)
        if "Adj Close" in data.columns:
              data.drop("Adj Close",axis=1,inplace=True)

        data = np.array(data)

        opens = []
        highs = []
        lows = []
        closes = []
        for ix in range(each_row_past, 1, -1):
            opens.append([data[i-ix][0] - data[i-ix+1][0] ])
            highs.append([data[i-ix][1] - data[i-ix+1][1] ])
            lows.append([data[i-ix][2] - data[i-ix+1][2] ])
            closes.append([data[i-ix][3] - data[i-ix+1][3] ])
        
        arr = np.concatenate((np.array(opens).flatten(), np.array(highs).flatten()
        , np.array(lows).flatten(), np.array(closes).flatten()), axis=0).reshape(-1, 1)
        arr = scaler(arr.reshape(-1, 1),min_range,max_range)
        mean_of_previous = []
        for ix in range(i-5,i,):
              mean_of_previous.append((data[ix][3]+data[ix][0])/2)
        mean_of_previous = sum(mean_of_previous)/5
        return arr,mean_of_previous
def make_prediction_for_yf(symbol,period,timeframe,index,min_range,max_range,each_row_past,interval_for_that):
    raw_data, mean_of_previous = process_for_prediction(
        yf.download(symbol, period=period, interval=timeframe), index,min_range,max_range,each_row_past,interval_for_that)
    return f"""YF   : {model.predict(np.array(raw_data).reshape(1,-1)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction_for_tv(symbol,exchange,timeframe,tindex,min_range,max_range,each_row_past,interval_for_that):
    tv = TvDatafeed()
    raw_data, mean_of_previous = process_for_prediction(
       tv.get_hist(symbol=symbol, exchange=exchange, interval=timeframe, n_bars=5000), tindex,min_range,max_range,each_row_past,interval_for_that)
    return f"""TVB  : {model.predict(np.array(raw_data).reshape(1,-1)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction(ysymbol,period,timeframe,tsymbol,texchange,ttimeframe,tindex,index):
  print(make_prediction_for_yf(ysymbol,period,timeframe,index,min_range,max_range,each_row_past,interval_for_that))
  print(make_prediction_for_tv(tsymbol,texchange,ttimeframe,tindex,min_range,max_range,each_row_past,interval_for_that))

In [ ]:
make_prediction("btc-usd","max","1d","btcusd","bitstamp",Interval.in_daily,-1,-1)

In [ ]:
yf.download("btc-usd",period="max",interval="1wk")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')